In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import config
import torch

/home/ubuntu/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(config.model_id)
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("<|finetune_right_pad_id|>")

In [3]:
model = AutoModelForCausalLM.from_pretrained(config.model_id, 
                                             quantization_config=config.bnb_config,  
                                             device_map='auto', 
                                             torch_dtype=torch.bfloat16)
model.config.pad_token_id = tokenizer.pad_token_id  

lora_model = PeftModel.from_pretrained(model, "model")

Loading checkpoint shards: 100%|██████████| 4/4 [01:08<00:00, 17.18s/it]


In [16]:
prompt = ["One wise man said:"]


In [58]:
inputs = tokenizer(prompt, return_tensors='pt', padding=True).to('cuda')
outputs = lora_model.generate(**inputs, do_sample=True, top_p = 0.9, max_new_tokens=64, num_return_sequences=10)
generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for gen in generated:
    print(gen)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


One wise man said: The only way to grow is to fall down.
One wise man said: When you think you can't, you can't. When you think you can, you can.
One wise man said: We are all the same. We are all children of the universe.
One wise man said: If you want to change your life, you must first change your mind.
One wise man said: The only way to make sense of change is to plunge into it, move with it, and join in the dauntless dance of change.
One wise man said: You must have the courage to follow your dreams.
One wise man said: If you love something, let it go. If it comes back to you, it's yours.
One wise man said: The greatest gift you can give to anyone is your time.
One wise man said: The heart is a muscle, but it needs love to keep it strong.
One wise man said: The only thing that can save us is the light of love.
